# 信号塔覆盖

## 目的和先决条件

在此示例中，我们将解决一个简单的覆盖问题：如何构建一个信号塔网络以提供信号覆盖给尽可能多的人。我们将构建业务问题的数学模型，然后用 Gurobi Python API 进行建模，并计算最佳解决方案。

该建模示例处于初级阶段，我们假设你了解Python，并且具有一些有关构建数学优化模型的知识。

**注意：** 你可以通过单击 [此处](https://github.com/arvinxx/gurobi-and-mathematical-modeling/archive/master.zip) 下载包含此示例和其他示例的代码。为了正确运行此 Jupyter Notebook，你必须具有 Gurobi 许可证。如果你没有，则可以**商业用户身份**申请 [试用许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/)，或以**学术用户身份**下载 [免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses)。

## 动机

在过去的十年里，智能手机彻底改变了我们的生活，远远超出了我们的沟通方式。除了打电话,发短信,发邮件,现在全世界有超过二十亿人使用这些设备导航到书出租车,比较产品评论和价格,遵循新闻,看电影,听音乐,玩游戏,拍照,参与社交媒体,和许多其他应用程序。

蜂窝网络是一种手持式智能手机网络，其中每部手机通过蜂窝基站(蜂窝基站)的本地天线通过无线电波与电话网络进行通信。一个重要的问题是如何安置手机信号塔，以便向最多的人提供信号覆盖。

## 问题描述

一家电信公司需要建造一组蜂窝塔，以为给定城市的居民提供信号覆盖范围。已经确定了可以建造塔楼的许多潜在位置。这些塔有固定的范围，并且-由于预算限制，只能建造数量有限的塔。鉴于这些限制，该公司希望将覆盖范围扩大到尽可能多的人口。为了简化问题，该公司将其希望覆盖的区域划分为一组区域，每个区域都有已知的人口。然后，目标是选择公司应在哪些潜在位置上建造手机信号塔，以便覆盖尽可能多的人。

蜂窝基站覆盖问题是最大覆盖位置问题[1]的一个实例。它也与集合覆盖问题（Set Cover Problem）有关。集合覆盖问题出现在许多不同的领域，非常重要的应用来自航空工业。例如，机组调度和航道分配问题[2]。

## 解决方法

数学优化（也称为数学编程）是一种声明性方法，其中建模者制定了一个优化问题，该问题捕获了复杂决策问题的关键特征。然后 Gurobi 使用最先进的数学和计算机科学技术来解决这一类数学优化问题。


数学优化模型具有五个组成部分：

* 数据集（Sets）
* 参数(Parameters)
* 决策变量(Decision variables)
* 约束（Constraints）
* 目标函数（Objective function(s)）

我们现在提出一个混合整数规划（MIP）模型来解决信号塔覆盖问题。

## 模型制定

### 索引和集合

$i \in T$: 信号塔候选地点的索引和集合。

$j \in R$: 区域的索引和集合。

$G(T,R,E)$: 在候选地点 $T$ 中定义的[二分图](https://zh.wikipedia.org/wiki/%E4%BA%8C%E5%88%86%E5%9B%BE)，我们要覆盖的区域集为 $R$，而 $E$ 是集合的另一组边。其中，如果中的区域 $j \in R$ 可以被位置为 $i \in T$ 的塔覆盖，则在 $(i,j) \in E$ 有一条边。

### 参数

$c_{i} \in \mathbb{R}^+$: 在站点 $i$ 建立塔的成本。

$p_{j} \in \mathbb{N}$: 区域$j$的人口。

### 决策变量

$covered_{j} \in \{0, 1 \}$: 如果覆盖区域 $j$，则值等于1；否则为0。


$build_{i} \in \{0, 1 \}$: 如果建造了塔 $i$，则此变量等于1，否则为0。


### 目标函数

- **人口覆盖**：我们力求使塔楼覆盖的总人口最大化。


\begin{equation}
\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}
\tag{0}
\end{equation}

### 约束

- **覆盖**：对于每个区域$j \in R$，确保至少有一个信号塔。

\begin{equation}
\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R
\tag{1}
\end{equation}

- **预算**：我们需要确保建造塔楼的总成本不超过指定的预算。

\begin{equation}
\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}
\tag{2}
\end{equation}

## Python 实现

本示例考虑了6个塔和9个区域的二分图。下表说明了每个手机信号塔站点（行）所覆盖的区域（列）。


| <i></i> | 地区0 | 地区1 | 地区2 | 地区3 | 地区4 | 地区5 | 地区6 | 地区7 | 地区8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |  --- |
| 塔0 | 1 | 1 | - | - | - | 1 | - | - |  - |
| 塔1 | 1 | - | - | - | - | - | - | 1 |  1 |
| 塔2 | - | - | 1 | 1 | 1 | - | 1 | - |  - |
| 塔3 | - | - | 1 | - | - | 1 | 1 | - |  - |
| 塔4 | 1 | - | 1 | - | - | - | 1 | 1 |  1 |
| 塔5 | - | - | - | 1 | 1 | - | - | - |  1 |

下表列出了每个地区的人口。

| <i></i> | 地区0 | 地区1 | 地区2 | 地区3 | 地区4 | 地区5 | 地区6 | 地区7 | 地区8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 人口 | 523 | 690 | 420 | 1010 | 1200 | 850 | 400 | 1008 | 950 |

在下表中列出了在每个地点建造基站塔的成本。

| <i></i> | 成本 (百万美元) |
| --- | --- |
| 塔0 | 4.2 |
| 塔1 | 6.1 |
| 塔2 | 5.2 |
| 塔3 | 5.5 |
| 塔4 | 4.8 |
| 塔5 | 9.2 | 

总预算为 $\$20,000,000$。

我们现在导入 Gurobi Python 模块。然后用给定的数据初始化数据结构。


In [7]:
import gurobipy as gp
from gurobipy import GRB

# 官方测试版本 Gurobi v9.0.0 & Python 3.7.0

# 译者测试版本 Gurobi v9.1.0 & Python 3.8.6

# 参数
budget = 20
regions, population = gp.multidict({
    0: 523, 1: 690, 2: 420,
    3: 1010, 4: 1200, 5: 850,
    6: 400, 7: 1008, 8: 950
})

sites, coverage, cost = gp.multidict({
    0: [{0,1,5}, 4.2],
    1: [{0,7,8}, 6.1],
    2: [{2,3,4,6}, 5.2],
    3: [{2,5,6}, 5.5],
    4: [{0,2,6,7,8}, 4.8],
    5: [{3,4,8}, 9.2]
})


### 模型开发

现在，我们通过定义决策变量、约束条件和目标函数来确定蜂窝基站覆盖问题的模型。然后进行优化过程，Gurobi 会得到建造信号塔的最佳方案（在给定的预算分配下，将人口覆盖范围最大化）。

In [8]:
# MIP  模型

m = gp.Model("cell_tower")

build = m.addVars(len(sites), vtype=GRB.BINARY, name="Build")
is_covered = m.addVars(len(regions), vtype=GRB.BINARY, name="Is_covered")

m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")
m.addConstr(build.prod(cost) <= budget, name="budget")

m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

m.optimize() 

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 10 rows, 15 columns and 36 nonzeros
Model fingerprint: 0xfa0fabb2
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 21 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 7.051000e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7051.0000000 7051.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.01 seconds
Thread coun

## 分析

优化模型的结果表明，$\$20,000,000$ 预算可以覆盖的最大人数为 7,051 人。让我们看看实现最佳结果的解决方案。

### 基站建造计划

此计划决定在该场地的哪个位置建造基站。

In [9]:
# 显示决策变量的最佳值

for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n 建造基站塔的位置为：{tower}。")


 建造基站塔的位置为：0.

 建造基站塔的位置为：2.

 建造基站塔的位置为：4.


### 需求完成指标


- **覆盖率**: 所建基站覆盖人口的百分比。

In [10]:
# 建造的信号塔所覆盖的人口百分比

total_population = 0

for region in range(len(regions)):
    total_population += population[region]

coverage = round(100*m.objVal/total_population, 2)

print(f"\n 与基站建设计划相关的人口覆盖率为： {coverage}%")


 与基站建设计划相关的人口覆盖率为： 100.0 %


### 资源利用率指标

- **预算消耗**: 占用于建造基站的总预算的百分比。

In [11]:
# 建造基站所需的预算百分比

total_cost = 0

for tower in range(len(sites)):
    if (abs(build[tower].x) > 0.5):
        total_cost += cost[tower]*int(build[tower].x)

budget_consumption = round(100*total_cost/budget, 2)

print(f"\n 与基站建造计划相关的预算消耗百分比为: {budget_consumption} %")


 与基站建造计划相关的预算消耗百分比为: 71.0 %


##  结论

在这个例子中，我们解决了建立基站塔以在满足预算限制的情况下为最多的人提供信号覆盖的问题。我们学习了如何将问题表示为一个 MIP 模型。此外，我们还学习了如何实现MIP模型公式，并使用 Gurobi 对其求解。

## 参考资料

[1] Richard Church and Charles R. Velle. "The Maximal Covering Location Problem". Papers in Regional Science, 1974, vol. 32, issue 1, 101-118.

[2] Tail Assignment Problem. https://www.gurobi.com/case_study/air-france-tail-assignment-optimization/

Copyright © 2020 Gurobi Optimization, LLC

翻译 By Arvin Xu